In [2]:
import cv2
import numpy as np

In [6]:
apple=cv2.imread('apple.jpg')
orange=cv2.imread('orange.jpg')

apple_orange=np.hstack((apple[:,:256], orange[:,256:]))
cv2.imshow('apple_orange', apple_orange)

cv2.waitKey(0)
cv2.destroyAllWindows()

## To clearly blend apple_orange image follow five steps
        load two images
        find gaussian pyramid for apple and orange
        from gaussian pyramid ,find their laplacian pyramid
        now join left half of the apple and orange in all levels of laplacian pyramid
        finally from this joint images pyramid reconstruct the original image

In [7]:
apple=cv2.imread('apple.jpg')
orange=cv2.imread('orange.jpg')

apple_orange=np.hstack((apple[:,:256], orange[:,256:]))

# generate gaussian pyramid
apple_copy=apple.copy()
gp_apple=[apple_copy]

for i in range(6):
    apple_copy=cv2.pyrDown(apple_copy)
    gp_apple.append(apple_copy)

orange_copy=orange.copy()
gp_orange=[orange_copy]

for i in range(6):
    orange_copy=cv2.pyrDown(orange_copy)
    gp_orange.append(orange_copy)
    

# generate laplacian pyramid
# a level in laplacian pyramid is formed by difference between that level and its extended version in gaussian pyramid
apple_copy=gp_apple[5]

lp_apple=[apple_copy]
for i in range(5,0,-1):
    gau_extended=cv2.pyrUp(gp_apple[i])
    laplacian =cv2.subtract(gp_apple[i-1], gau_extended)
    lp_apple.append(laplacian)

orange_copy=gp_orange[5]

lp_orange=[orange_copy]
for i in range(5,0,-1):
    gau_extended=cv2.pyrUp(gp_orange[i])
    laplacian =cv2.subtract(gp_orange[i-1], gau_extended)
    lp_orange.append(laplacian)



# now join half of apple and orange from their laplace pyramids
apple_orange_pyramid=[]
n=0
for apple_lap, orange_lap in zip(lp_apple, lp_orange):
    n+=1
    cols, rows, ch =apple_lap.shape
    laplacian=np.hstack((apple_lap[:,0:int(cols//2)], orange_lap[:,int(cols//2):]))
    apple_orange_pyramid.append(laplacian)
    
    
#now reconstruct
apple_orange_reconstruct=apple_orange_pyramid[0]
for i in range(1,6):
    apple_orange_reconstruct=cv2.pyrUp(apple_orange_reconstruct)
    apple_orange_reconstruct=cv2.add(apple_orange_pyramid[i],apple_orange_reconstruct)
    
    
cv2.imshow('apple_orange_reconstruct', apple_orange_reconstruct)


cv2.waitKey(0)
cv2.destroyAllWindows()

## we can compare the results both apple_orange and apple_orange_reconstruct :
## apple_orange_reconstruct- images are blends perfectly